#### Implementing Feature Union
<div>Using the given custom transformer, StartingVerbExtractor, add a feature union to your pipeline to incorporate a feature that indicates with a boolean value whether the starting token of a post is identified as a verb.</div>

In [1]:
import nltk
# nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

In [3]:
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from Custom_Transformer import StartingVerbExtractor

In [5]:
def model_pipeline():
    pipeline = Pipeline([
        ('feature_union',FeatureUnion([
         ('nlp_pipeline',Pipeline([
             ('vec',CountVectorizer(tokenizer=tokenize)),
             ('tfidf',TfidfTransformer())
         ])),
         ('verb_extractor',StartingVerbExtractor())
        ])),
        ('clf_rf',RandomForestClassifier())    
    ])
     
    return pipeline

#### Run Program to Test

In [6]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [7]:
def load_data():
    df = pd.read_csv('corporate_messaging.csv', encoding='latin-1')
    df = df[(df["category:confidence"] == 1) & (df['category'] != 'Exclude')]
    X = df.text.values
    y = df.category.values
    return X, y


def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


def display_results(y_input, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_input, y_pred, labels=labels)
    accuracy = (y_pred == y_input).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    print('')


def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model = model_pipeline()
    model.fit(X_train, y_train)
    
    y_pred_train = model.predict(X_train)

    display_results(y_train, y_pred_train)
    
    y_pred = model.predict(X_test)

    display_results(y_test, y_pred)

main()

Labels: ['Action' 'Dialogue' 'Information']
Confusion Matrix:
 [[ 333    0    0]
 [   0   89    0]
 [   0    0 1380]]
Accuracy: 1.0

Labels: ['Action' 'Dialogue' 'Information']
Confusion Matrix:
 [[ 94   0  29]
 [  1  30   4]
 [  4   1 438]]
Accuracy: 0.9351081530782029

